<a href="https://colab.research.google.com/github/matari18/ml/blob/main/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [54]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [55]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [56]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [57]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [58]:
data.describe(include = 'object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [59]:
# ваш код здесь
data['Marital Status'] = data['Marital Status'].map({'Married' : 1, 'Single' : 0})
data['Gender'] = data['Gender'].map({'Male' : 1, 'Female' : 0})
data['Home Owner'] = data['Home Owner'].map({'Yes' : 1, 'No' : 0})
data['Purchased Bike'] = data['Purchased Bike'].map({'No' : 1, 'Yes' : 0})

Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [60]:
# ваш код здесь
data = data.drop(columns=['Education', 'Occupation', 'Commute Distance', 'Region'])

Удалите столбец `ID`, так как он по сути нечисловой.

In [61]:
# ваш код здесь
data = data.drop(columns=['ID'])

Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [62]:
# ваш код здесь
X = data.drop('Purchased Bike', axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [63]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [64]:
Xtrain.shape

(700, 7)

In [65]:
Xtrain.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
541,1,1,50000,1,1,0,34
440,0,1,90000,0,0,3,34
482,0,1,40000,0,0,0,36
422,0,1,110000,0,0,3,32
778,1,1,90000,5,1,3,41


Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [66]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

Xtrain = pd.DataFrame(data=Xtrain_scaled, columns=Xtrain.columns.tolist())
Xtest = pd.DataFrame(data=Xtest_scaled, columns=Xtest.columns.tolist())

Теперь обучите логистическую регрессию на тренировочных данных

In [69]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [77]:
# ваш код здесь
prediction_train = lr.predict(Xtrain)
prediction_test = lr.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [80]:
from sklearn.metrics import accuracy_score

accuracy_train = accuracy_score(ytrain, prediction_train)
accuracy_test = accuracy_score(ytest, prediction_test)
accuracy_train, accuracy_test

(0.6342857142857142, 0.58)

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [89]:
from sklearn.preprocessing import PolynomialFeatures

#pf = PolynomialFeatures(2)
#pf.fit(Xtrain, ytrain)
#Xtrain_pf = pf.transform(Xtrain)
#Xtest_pf = pf.transform(Xtest)

pf = PolynomialFeatures(degree=2).fit(Xtrain)
#print p.get_feature_names(data.columns)
pf.get_feature_names(Xtrain.columns)


#Xtrain = pd.DataFrame(columns = list(Xtrain.columns+p.), data = Xtrain_pf)
#Xtest = pd.DataFrame(columns = list(Xtest.columns), data = Xtest_pf)

AttributeError: 'PolynomialFeatures' object has no attribute 'get_feature_names'

In [88]:
Xtrain_pf.get_feature_names(Xtrain.columns)

AttributeError: 'numpy.ndarray' object has no attribute 'get_feature_names'

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [73]:
# ваш код здесь

**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [74]:
# ваш код здесь

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [75]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 7), (210, 7))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [76]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    # ваш код здесь

print (BestThr, RecMax, BestAcc)

IndentationError: expected an indented block after 'for' statement on line 7 (<ipython-input-76-a099206b1607>, line 10)

Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [ ]:
# ваш код здесь

**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.